In [ ]:
!pip install swifter
!pip install flair

In [ ]:

import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from html import unescape
import math
import csv


filename = r"/content/drive/MyDrive/machine_learning/UkraineCombinedTweetsDeduped_FEB27.csv.gzip"
data = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)


data.shape




oneFifth = math.ceil(len(data) * 0.01)



data = data[data["language"] == "en"].sample(oneFifth)

data.reset_index(drop=True, inplace=True)



data.shape





def remove_urls(z):
    cleaned_string = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', str(z), flags=re.MULTILINE)
    return cleaned_string



def unescape_stuff(z):
    soup = BeautifulSoup(unescape(z), 'lxml')
    return soup.text



def deEmojify(z):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', z)



def unify_whitespaces(z):
    cleaned_string = re.sub(' +', ' ', z)
    return cleaned_string



def remove_symbols(z):
    cleaned_string = re.sub(r"[^a-zA-Z0-9?!.,]+", ' ', z)
    return cleaned_string



data['text'] = data['text'].str.lower()
data['text'] = data['text'].apply(remove_urls)
data['text'] = data['text'].apply(unescape_stuff)
data['text'] = data['text'].apply(deEmojify)
data['text'] = data['text'].apply(remove_symbols)
data['text'] = data['text'].apply(unify_whitespaces)



data['text'].head()




from flair.models import TextClassifier
from flair.data import Sentence

sia = TextClassifier.load('en-sentiment')


def flair_prediction(z):

    sentence = Sentence(z)

    try:
        sia.predict(sentence)
        score = sentence.labels[0]
        staging_score = str(score).replace("(",",").replace(")","")

        sentiment_score = staging_score.split(",")

        if "POSITIVE" in str(sentiment_score[0]):
            return "POSITIVE", float(sentiment_score[1].strip())
        elif "NEGATIVE" in str(sentiment_score[0]):
            return "NEGATIVE", float(sentiment_score[1].strip())
        else:
            return "NEUTRAL", 0.00
    except Exception:
        print(sentence)
        pass

    return "ERROR",0.00



data['Sentiment'] = ""
data['Sentiment_Score'] = np.nan



import swifter
data["Sentiment"],data["Sentiment_Score"] =  zip(*data["text"].swifter.apply(flair_prediction))




<ipython-input-22-43c7aff12ca9>:49: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(unescape(x), 'lxml')


Pandas Apply:   0%|          | 0/3571 [00:00<?, ?it/s]

"## Flair Sentiment Is Very Slow But Promises Better Performance than Rule-based Sentiment Analysis Libraries, if you sentiment analysis extraction is time-critical, dont use this\n\n## Let's take a peek at the top 5 records/ rows\n"

In [ ]:
data.head().T





,0,1,2,3,4
userid,572394074,1232695442,833065841646252034,823870827791785984,1281303947857113088
username,pravinjasmaju,areiasgabriel,daddafelix,UncleJoeGTA,VickiMo08056436
acctdesc,NaN,"Rj, 23.",|| Christ's ✝️❤\n|| I ❤ Mahoney 👩‍🍼 😍🥰\n|| Ibi...,Author. I make the impossible look easy. Numbe...,A daughter. A sister. A mother. A granny. A gr...
location,"Malacca, Malaysia",Rio De Janeiro,Nigeria,"Toronto, Ontario",NaN
following,1328,230,370,7197,3901
followers,460,325,404,7374,1039
totaltweets,38494,15946,4212,25206,231054
usercreatedts,2012-05-06 07:21:24.000000,2013-03-02 03:26:49.000000,2017-02-18 21:29:09.000000,2017-01-24 12:31:26.000000,2020-07-09 19:08:12.000000
tweetid,1497741773770166273,1497736608023556098,1497737455243571201,1497781718073421827,1497751456325242888
tweetcreatedts,2022-02-27 01:14:00,2022-02-27 00:53:28,2022-02-27 00:56:50,2022-02-27 03:52:43,2022-02-27 01:52:28


In [ ]:
data.groupby(["Sentiment"]).agg({'Sentiment_Score': ['count','mean']})

Sentiment_Score          
                    count      mean
Sentiment                          
NEGATIVE             1448  0.940230
NEUTRAL              1611  0.000000
POSITIVE              512  0.887229

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

label_encoder = LabelEncoder()


data['Sentiment_encoded'] = label_encoder.fit_transform(data['Sentiment'])


y = data['Sentiment_encoded']
X = data['Sentiment_Score'].values.reshape(-1, 1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


svm_classifier = SVC(kernel='linear')


svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)
print(">>>>")
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Classification Report:
              precision    recall  f1-score   support

           0       0.75      1.00      0.85       288
           1       1.00      1.00      1.00       329
           2       0.00      0.00      0.00        98

    accuracy                           0.86       715
   macro avg       0.58      0.67      0.62       715
weighted avg       0.76      0.86      0.80       715

>>>>
Accuracy: 0.862937062937063


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
